In [1]:
import os
import glob
import hdf5_getters
import time

import pandas as pd
import numpy as np
from scipy import stats

In [2]:
h5 = hdf5_getters.open_h5_file_read('./MillionSongSubset/AdditionalFiles/subset_msd_summary_file.h5')

In [3]:
h5.root

/ (RootGroup) 'H5 Song File'
  children := ['musicbrainz' (Group), 'analysis' (Group), 'metadata' (Group)]

In [4]:
h5.root.musicbrainz

/musicbrainz (Group) 'data about the song coming from MusicBrainz'
  children := ['songs' (Table)]

In [5]:
h5.root.musicbrainz.songs

/musicbrainz/songs (Table(10000,), shuffle, zlib(1)) 'table of data coming from MusicBrainz'
  description := {
  "idx_artist_mbtags": Int32Col(shape=(), dflt=0, pos=0),
  "year": Int32Col(shape=(), dflt=0, pos=1)}
  byteorder := 'little'
  chunkshape := (1024,)

In [6]:
h5.root.metadata

/metadata (Group) 'metadata about the song'
  children := ['songs' (Table)]

In [7]:
h5.root.metadata.songs

/metadata/songs (Table(10000,), shuffle, zlib(1)) 'table of metadata for one song'
  description := {
  "analyzer_version": StringCol(itemsize=32, shape=(), dflt='', pos=0),
  "artist_7digitalid": Int32Col(shape=(), dflt=0, pos=1),
  "artist_familiarity": Float64Col(shape=(), dflt=0.0, pos=2),
  "artist_hotttnesss": Float64Col(shape=(), dflt=0.0, pos=3),
  "artist_id": StringCol(itemsize=32, shape=(), dflt='', pos=4),
  "artist_latitude": Float64Col(shape=(), dflt=0.0, pos=5),
  "artist_location": StringCol(itemsize=1024, shape=(), dflt='', pos=6),
  "artist_longitude": Float64Col(shape=(), dflt=0.0, pos=7),
  "artist_mbid": StringCol(itemsize=40, shape=(), dflt='', pos=8),
  "artist_name": StringCol(itemsize=1024, shape=(), dflt='', pos=9),
  "artist_playmeid": Int32Col(shape=(), dflt=0, pos=10),
  "genre": StringCol(itemsize=1024, shape=(), dflt='', pos=11),
  "idx_artist_terms": Int32Col(shape=(), dflt=0, pos=12),
  "idx_similar_artists": Int32Col(shape=(), dflt=0, pos=13),
  "relea

In [8]:
h5.root.analysis

/analysis (Group) 'Echo Nest analysis of the song'
  children := ['songs' (Table)]

In [9]:
h5.root.analysis.songs

/analysis/songs (Table(10000,), shuffle, zlib(1)) 'table of Echo Nest analysis for one song'
  description := {
  "analysis_sample_rate": Int32Col(shape=(), dflt=0, pos=0),
  "audio_md5": StringCol(itemsize=32, shape=(), dflt='', pos=1),
  "danceability": Float64Col(shape=(), dflt=0.0, pos=2),
  "duration": Float64Col(shape=(), dflt=0.0, pos=3),
  "end_of_fade_in": Float64Col(shape=(), dflt=0.0, pos=4),
  "energy": Float64Col(shape=(), dflt=0.0, pos=5),
  "idx_bars_confidence": Int32Col(shape=(), dflt=0, pos=6),
  "idx_bars_start": Int32Col(shape=(), dflt=0, pos=7),
  "idx_beats_confidence": Int32Col(shape=(), dflt=0, pos=8),
  "idx_beats_start": Int32Col(shape=(), dflt=0, pos=9),
  "idx_sections_confidence": Int32Col(shape=(), dflt=0, pos=10),
  "idx_sections_start": Int32Col(shape=(), dflt=0, pos=11),
  "idx_segments_confidence": Int32Col(shape=(), dflt=0, pos=12),
  "idx_segments_loudness_max": Int32Col(shape=(), dflt=0, pos=13),
  "idx_segments_loudness_max_time": Int32Col(shape=()

In [10]:
tids = [(row['track_id'], row['tempo'], row['loudness']) for row in h5.root.analysis.songs.where('tempo>245')]
#'(tempo>245) & (tempo<250)')]
tids

[('TRAFJKG128F9332800', 246.5, -11.906),
 ('TRAZFZS128F4272A31', 246.593, -22.28),
 ('TRAGYJM128EF3466D2', 253.357, -3.868),
 ('TRBHZRN128F92EFF91', 258.677, -14.165),
 ('TRBFXLH12903CBAFAD', 248.079, -16.386),
 ('TRBGYHC12903D0626A', 262.828, -11.534)]

In [105]:
tids_titles = []

def get_items(t_id, titles, metas, sh5):
    title = hdf5_getters.get_title(sh5)
    artist = hdf5_getters.get_artist_name(sh5)
    artist_id = hdf5_getters.get_artist_id(sh5)
    song_id = hdf5_getters.get_song_id(sh5)
    titles.append((t_id, title, artist_id, artist, song_id))
    song_hottt = hdf5_getters.get_song_hotttnesss(sh5)
    artist_hottt = hdf5_getters.get_artist_hotttnesss(sh5)
    artist_fam = hdf5_getters.get_artist_familiarity(sh5)
    metas.append((t_id, song_hottt, artist_hottt, artist_fam))
                    
def get_matching_titles(track_ids=None, basedir='./MillionSongSubset/data/',ext='.h5', verbose=0):
    t1 = time.time()
    titles = []
    metas = []
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        for f in files:
            if track_ids and len(titles) == len(track_ids):
                break
            sh5 = hdf5_getters.open_h5_file_read(f)
            if track_ids:
                for t_id in track_ids:
                    if hdf5_getters.get_track_id(sh5) == t_id:
                        get_items(t_id, titles, metas, sh5)
                        break
            else:
                t_id = hdf5_getters.get_track_id(sh5)
                get_items(t_id, titles, metas, sh5)
            sh5.close()
    t2 = time.time()
    if verbose == 1:
        print "total time: ", (t2-t1)
    return titles, metas

tids_only = [t[0] for t in tids]
titles, metas = get_matching_titles(verbose=1)#track_ids=tids_only, verbose=1)
print titles
print metas

total time:  66.7843520641
([('TRBGYHC12903D0626A', 'Endless Light', 'AR0JEOI1187B98B918', 'Erik Berglund', 'SODWYEY12AC468C30F'), ('TRBHZRN128F92EFF91', 'Kapitel 4', 'ARIEIJW1187B98F6DD', 'Oliver Kalkofe', 'SOQFMJN12AB0181D3D'), ('TRBFXLH12903CBAFAD', "Two of a Kind_ Workin' on a Full House", 'ARBMHBC11F4C8403A2', 'Hushabye Baby', 'SOOMCCM12AB01896EA'), ('TRAZFZS128F4272A31', 'Severe Severing', 'ARSL3N21187B98DFC5', 'Klaus Badelt', 'SOABYBF12A8C138A09'), ('TRAGYJM128EF3466D2', 'Been There All The Time', 'AR9YWMS1187FB43A34', 'Dinosaur Jr.', 'SOZRIFJ12A67ADA4BE'), ('TRAFJKG128F9332800', 'Ego is the drug/3am', 'ARCRSUM1187FB52AB0', 'The Frequency', 'SONNXDI12AB0186816')], [('TRBGYHC12903D0626A', 0.0, 0.33619115568493391, 0.37703408512839853), ('TRBHZRN128F92EFF91', 0.21508031850922793, 0.40619874783410698, 0.45248343829809812), ('TRBFXLH12903CBAFAD', nan, 0.36551400326265299, 0.35342955022197026), ('TRAZFZS128F4272A31', 0.32773668317784083, 0.46934037425924607, 0.68800334883697312), ('T

In [4]:
#seg_loud_max = [(row['track_id'], row['idx_sections_start']) for row in h5.root.analysis.songs.where('(tempo>238) & (tempo<240)')]
#print seg_loud_max

In [11]:
h52 = hdf5_getters.open_h5_file_read('./MillionSongSubset/data/A/F/J/TRAFJKG128F9332800.h5')#TRAGYJM128EF3466D2.h5')

In [12]:
h52.root.analysis

/analysis (Group) 'Echo Nest analysis of the song'
  children := ['beats_start' (EArray), 'segments_pitches' (EArray), 'segments_confidence' (EArray), 'sections_start' (EArray), 'tatums_confidence' (EArray), 'segments_timbre' (EArray), 'segments_loudness_start' (EArray), 'sections_confidence' (EArray), 'bars_confidence' (EArray), 'segments_loudness_max_time' (EArray), 'bars_start' (EArray), 'tatums_start' (EArray), 'beats_confidence' (EArray), 'segments_start' (EArray), 'segments_loudness_max' (EArray), 'songs' (Table)]

In [ ]:
def get_pitch_features(sh5):
    seg_pitches = hdf5_getters.get_segments_pitches(sh5)
    pitch_means = np.mean(seg_pitches, axis=0).tolist()
    pitch_medians = np.median(seg_pitches, axis=0).tolist()
    pitch_modes = stats.mode(seg_pitches)[0].tolist()[0]
    return pitch_means, pitch_medians, pitch_modes

def get_timbre_features(sh5):
    seg_timbres = hdf5_getters.get_segments_timbre(sh5)
    timbre_means = np.mean(seg_timbres, axis=0).tolist()
    timbre_medians = np.median(seg_timbres, axis=0).tolist()
    timbre_modes = stats.mode(seg_timbres)[0].tolist()[0]
    return timbre_means, timbre_medians, timbre_modes

def get_loudness_features(sh5):
    loud_maxs = hdf5_getters.get_segments_loudness_max(sh5)
    loud_mean = np.mean(loud_maxs)
    loud_median = np.median(loud_maxs)
    loud_mode = stats.mode(loud_maxs)[0].tolist()[0]
    return loud_mean, loud_median, loud_mode

def get_all_features(t_id, features, sh5):
    pitch_means, pitch_medians, pitch_modes = get_pitch_features(sh5)
    pitches = pitch_means + pitch_medians + pitch_modes
    loud_mean, loud_median, loud_mode = get_loudness_features(sh5)
    louds = [loud_mean, loud_median, loud_mode]
    timbre_means, timbre_medians, timbre_modes = get_timbre_features(sh5)
    timbres = timbre_means + timbre_medians + timbre_modes
    duration = hdf5_getters.get_duration(sh5)
    tempo = hdf5_getters.get_tempo(sh5)
    energy = hdf5_getters.get_energy(sh5)
    danceability = hdf5_getters.get_danceability(sh5)
    loudness = hdf5_getters.get_loudness(sh5)
    mode = hdf5_getters.get_mode(sh5)
    key = hdf5_getters.get_key(sh5)
    time_sig = hdf5_getters.get_time_signature(sh5)
    fade_in = hdf5_getters.get_end_of_fade_in(sh5)
    features.append([t_id, duration, fade_in, tempo, energy, danceability, loudness,
                     mode, key, time_sig]+pitches+louds+timbres)
                        
def get_track_features(track_ids=None, basedir='./MillionSongSubset/data/', ext='.h5', verbose=0):
    t1 = time.time()
    #pitches = [0,0,0,0,0,0,0,0,0,0,0,0]
    #louds = [0,0,0]
    #timbres = [0,0,0,0,0,0,0,0,0,0,0,0]
    count = 0
    features = []
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        for f in files:
            if track_ids and len(features) >= len(track_ids):
                break
            pitches = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
            louds = [0,0,0]
            timbres = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
            sh5 = hdf5_getters.open_h5_file_read(f)
            if track_ids:
                for t_id in track_ids:
                    if hdf5_getters.get_track_id(sh5) == t_id:
                        get_all_features(t_id, features, sh5)
            else:
                t_id = hdf5_getters.get_track_id(sh5)
                #pitch_means, pitch_medians, pitch_modes = get_pitch_features(sh5)
                get_all_features(t_id, features, sh5)
            if verbose > 1:
                if count >= 1000:
                    print "1000!"
                    count = 0
                else:
                    count += 1
            sh5.close()
    t2 = time.time()
    if verbose > 0:
        print "total time: ", (t2-t1)
    return features

#track_id, duration, fade_in, tempo, energy, danceability, loudness, mode, key, time_sig, pitches, louds, timbres
tids_only0 = [t[0] for t in tids]
#print tids_only0
print "track_id, duration, fade_in, tempo, energy, danceability, loudness, mode, key, time_sig, pitches(3x12), louds(3), timbres(3x12)"
features = get_track_features(verbose=2)#track_ids=tids_only0, verbose=1)

In [14]:
a = np.array([[1, 2], [3, 4], [5, 6]])
print np.mean(a, axis=0)
print np.mean(a, axis=1)

[ 3.  4.]
[ 1.5  3.5  5.5]


In [166]:
print seg_pitches.shape
print type(seg_pitches)
seg_pitch_mean = np.mean(seg_pitches, axis=0)
print seg_pitch_mean.shape, "\n", tuple(seg_pitch_mean)
print seg_pitches
print
print tuple(np.median(seg_pitches, axis=0))
print
print tuple(stats.mode(seg_pitches)[0].tolist()[0])


(530, 12)
<type 'numpy.ndarray'>
(12,) 
(0.46639056603773582, 0.55831886792452867, 0.59365471698113281, 0.3651773584905661, 0.42963396226415096, 0.33559433962264151, 0.38138679245283025, 0.33712452830188638, 0.44591320754717012, 0.68786603773584898, 0.42415094339622633, 0.48663962264150951)
[[ 0.833  0.333  0.224 ...,  0.614  1.     0.189]
 [ 0.299  0.195  0.228 ...,  0.73   0.423  0.217]
 [ 0.333  0.387  0.364 ...,  0.971  0.488  0.317]
 ..., 
 [ 0.112  0.167  1.    ...,  0.483  0.104  0.182]
 [ 0.536  1.     0.764 ...,  0.52   0.189  0.16 ]
 [ 0.07   0.077  0.547 ...,  1.     0.332  0.239]]

(0.42949999999999999, 0.502, 0.5555000000000001, 0.35149999999999998, 0.38700000000000001, 0.314, 0.32750000000000001, 0.29449999999999998, 0.40500000000000003, 0.74849999999999994, 0.39700000000000002, 0.443)

(1.0, 1.0, 1.0, 0.034, 1.0, 0.318, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0)


In [15]:
p_mean_titles = ["pitch_mean"+str(n) for n in range(1,13)]
p_median_titles = ["pitch_median"+str(n) for n in range(1,13)]
p_mode_titles = ["pitch_mode"+str(n) for n in range(1,13)]
print p_mean_titles

t_mean_titles = ["timbre_mean"+str(n) for n in range(1,13)]
t_median_titles = ["timbre_median"+str(n) for n in range(1,13)]
t_mode_titles = ["timbre_mode"+str(n) for n in range(1,13)]

['pitch_mean1', 'pitch_mean2', 'pitch_mean3', 'pitch_mean4', 'pitch_mean5', 'pitch_mean6', 'pitch_mean7', 'pitch_mean8', 'pitch_mean9', 'pitch_mean10', 'pitch_mean11', 'pitch_mean12']


In [16]:
heads = ["track_id", "duration", "fade_in", "tempo", "energy", "danceability", "loudness", 
         "mode", "key", "time_sig"] + p_mean_titles + p_median_titles + p_mode_titles + \
        ["loud_mean", "loud_median", "loud_mode"] \
         + t_mean_titles + t_median_titles + t_mode_titles
print len(heads), "\n", heads

85 
['track_id', 'duration', 'fade_in', 'tempo', 'energy', 'danceability', 'loudness', 'mode', 'key', 'time_sig', 'pitch_mean1', 'pitch_mean2', 'pitch_mean3', 'pitch_mean4', 'pitch_mean5', 'pitch_mean6', 'pitch_mean7', 'pitch_mean8', 'pitch_mean9', 'pitch_mean10', 'pitch_mean11', 'pitch_mean12', 'pitch_median1', 'pitch_median2', 'pitch_median3', 'pitch_median4', 'pitch_median5', 'pitch_median6', 'pitch_median7', 'pitch_median8', 'pitch_median9', 'pitch_median10', 'pitch_median11', 'pitch_median12', 'pitch_mode1', 'pitch_mode2', 'pitch_mode3', 'pitch_mode4', 'pitch_mode5', 'pitch_mode6', 'pitch_mode7', 'pitch_mode8', 'pitch_mode9', 'pitch_mode10', 'pitch_mode11', 'pitch_mode12', 'loud_mean', 'loud_median', 'loud_mode', 'timbre_mean1', 'timbre_mean2', 'timbre_mean3', 'timbre_mean4', 'timbre_mean5', 'timbre_mean6', 'timbre_mean7', 'timbre_mean8', 'timbre_mean9', 'timbre_mean10', 'timbre_mean11', 'timbre_mean12', 'timbre_median1', 'timbre_median2', 'timbre_median3', 'timbre_median4', 'timb

In [17]:
df = pd.DataFrame(features, columns=heads)

In [18]:
df.head(6)

,track_id,duration,fade_in,tempo,energy,danceability,loudness,mode,key,time_sig,...,timbre_mode3,timbre_mode4,timbre_mode5,timbre_mode6,timbre_mode7,timbre_mode8,timbre_mode9,timbre_mode10,timbre_mode11,timbre_mode12
0,TRBGYHC12903D0626A,1815.22240,0.231,262.828,0,0,-11.534,1,0,7,...,45.099,-15.044,-28.625,-48.740,2.410,-28.339,-4.786,-9.531,-4.447,20.122
1,TRBHZRN128F92EFF91,549.30240,0.334,258.677,0,0,-14.165,0,6,4,...,-109.044,-93.669,-80.048,-26.371,-77.829,-50.813,-45.996,-17.967,-34.878,22.613
2,TRBFXLH12903CBAFAD,224.05179,0.223,248.079,0,0,-16.386,0,11,4,...,-133.279,-57.786,38.605,-95.978,2.079,-30.368,9.187,0.498,-6.431,-0.419
3,TRAZFZS128F4272A31,592.56118,3.135,246.593,0,0,-22.280,0,9,4,...,9.628,-42.006,-59.177,-24.813,-20.528,-11.116,6.472,-17.506,-7.771,-21.699
4,TRAGYJM128EF3466D2,220.70812,0.136,253.357,0,0,-3.868,1,2,4,...,30.160,-0.286,-44.318,-36.757,-21.785,5.373,12.090,-0.733,5.899,-2.619
5,TRAFJKG128F9332800,1054.37995,0.207,246.500,0,0,-11.906,1,9,4,...,-65.493,-43.786,-23.578,-14.494,10.262,-12.958,-3.601,-11.768,1.496,-13.351


In [19]:
track_ids = df["track_id"]
track_ids.head(6)

0    TRBGYHC12903D0626A
1    TRBHZRN128F92EFF91
2    TRBFXLH12903CBAFAD
3    TRAZFZS128F4272A31
4    TRAGYJM128EF3466D2
5    TRAFJKG128F9332800
Name: track_id, dtype: object

In [20]:
modes = df["mode"]
modes.head(6)

0    1
1    0
2    0
3    0
4    1
5    1
Name: mode, dtype: int64

In [21]:
keys = pd.get_dummies(df["key"])
keys.head(6)

,0,2,6,9,11
0,1,0,0,0,0
1,0,0,1,0,0
2,0,0,0,0,1
3,0,0,0,1,0
4,0,1,0,0,0
5,0,0,0,1,0


In [22]:
times = pd.get_dummies(df["time_sig"]) #"ts-1", "ts1", "ts3", "ts4", "ts5", "ts6", "ts7"
times.head(6)

,4,7
0,0,1
1,1,0
2,1,0
3,1,0
4,1,0
5,1,0


In [23]:
categoricals = pd.concat([modes, keys,times], axis=1)
categoricals.head(6)

,mode,0,2,6,9,11,4,7
0,1,1,0,0,0,0,0,1
1,0,0,0,1,0,0,1,0
2,0,0,0,0,0,1,1,0
3,0,0,0,0,1,0,1,0
4,1,0,1,0,0,0,1,0
5,1,0,0,0,1,0,1,0


In [24]:
firsts = df.iloc[:,1:7]
firsts.head(6)

,duration,fade_in,tempo,energy,danceability,loudness
0,1815.22240,0.231,262.828,0,0,-11.534
1,549.30240,0.334,258.677,0,0,-14.165
2,224.05179,0.223,248.079,0,0,-16.386
3,592.56118,3.135,246.593,0,0,-22.280
4,220.70812,0.136,253.357,0,0,-3.868
5,1054.37995,0.207,246.500,0,0,-11.906


In [28]:
firsts_zscore = pd.DataFrame()
for f in firsts:
    f_zscore = f + '_zscore'
    if firsts[f].std(ddof=0) == 0:
        firsts_zscore[f_zscore] = 0.0
    else:
        firsts_zscore[f_zscore] = (firsts[f] - firsts[f].mean())/firsts[f].std(ddof=0)
firsts_zscore.fillna(0)
firsts_zscore.head(6)

,duration_zscore,fade_in_zscore,tempo_zscore,energy_zscore,danceability_zscore,loudness_zscore
0,1.932092,-0.442154,1.619931,0,0,0.328287
1,-0.348405,-0.347275,0.957805,0,0,-0.145635
2,-0.934328,-0.449523,-0.732683,0,0,-0.545705
3,-0.270476,2.232878,-0.969715,0,0,-1.607392
4,-0.940352,-0.529664,0.109211,0,0,1.709166
5,0.561469,-0.464262,-0.984549,0,0,0.261279


In [29]:
lasts = df.iloc[:,10:]
lasts.head(6)

,pitch_mean1,pitch_mean2,pitch_mean3,pitch_mean4,pitch_mean5,pitch_mean6,pitch_mean7,pitch_mean8,pitch_mean9,pitch_mean10,...,timbre_mode3,timbre_mode4,timbre_mode5,timbre_mode6,timbre_mode7,timbre_mode8,timbre_mode9,timbre_mode10,timbre_mode11,timbre_mode12
0,0.406870,0.116808,0.336348,0.102231,0.429092,0.247032,0.108656,0.395560,0.090663,0.298550,...,45.099,-15.044,-28.625,-48.740,2.410,-28.339,-4.786,-9.531,-4.447,20.122
1,0.423766,0.459516,0.423902,0.402862,0.409096,0.430284,0.450812,0.413044,0.409740,0.407291,...,-109.044,-93.669,-80.048,-26.371,-77.829,-50.813,-45.996,-17.967,-34.878,22.613
2,0.129961,0.187738,0.053969,0.117769,0.493928,0.101809,0.271167,0.196369,0.144160,0.168450,...,-133.279,-57.786,38.605,-95.978,2.079,-30.368,9.187,0.498,-6.431,-0.419
3,0.304885,0.275846,0.406555,0.296974,0.329690,0.240757,0.206145,0.292987,0.172487,0.282570,...,9.628,-42.006,-59.177,-24.813,-20.528,-11.116,6.472,-17.506,-7.771,-21.699
4,0.466391,0.558319,0.593655,0.365177,0.429634,0.335594,0.381387,0.337125,0.445913,0.687866,...,30.160,-0.286,-44.318,-36.757,-21.785,5.373,12.090,-0.733,5.899,-2.619
5,0.174128,0.238498,0.397535,0.235221,0.533889,0.132505,0.150335,0.127786,0.238987,0.208749,...,-65.493,-43.786,-23.578,-14.494,10.262,-12.958,-3.601,-11.768,1.496,-13.351


In [30]:
lasts_zscore = pd.DataFrame()
for l in lasts:
    l_zscore = l + '_zscore'
    if lasts[l].std(ddof=0) == 0:
        lasts_zscore[l_zscore] = 0.0
    else:
        lasts_zscore[l_zscore] = (lasts[l] - lasts[l].mean())/lasts[l].std(ddof=0)
lasts_zscore.fillna(0)
lasts_zscore.head(6)

,pitch_mean1_zscore,pitch_mean2_zscore,pitch_mean3_zscore,pitch_mean4_zscore,pitch_mean5_zscore,pitch_mean6_zscore,pitch_mean7_zscore,pitch_mean8_zscore,pitch_mean9_zscore,pitch_mean10_zscore,...,timbre_mode3_zscore,timbre_mode4_zscore,timbre_mode5_zscore,timbre_mode6_zscore,timbre_mode7_zscore,timbre_mode8_zscore,timbre_mode9_zscore,timbre_mode10_zscore,timbre_mode11_zscore,timbre_mode12_zscore
0,0.700365,-1.228521,-0.200384,-1.322970,-0.131010,-0.008590,-1.253652,0.989001,-1.197736,-0.254276,...,1.189369,0.902684,0.114042,-0.282640,0.677379,-0.392908,-0.017701,-0.004100,0.248583,1.196521
1,0.833026,0.995437,0.342574,1.308514,-0.440552,1.623252,1.554294,1.158948,1.195883,0.378513,...,-1.039499,-1.720896,-1.271733,0.555001,-2.043612,-1.660007,-2.119900,-1.163327,-2.084980,1.350574
2,-1.473743,-0.768234,-1.951518,-1.186964,0.872681,-1.301790,0.080016,-0.947156,-0.796422,-1.011364,...,-1.389931,-0.523542,1.925792,-2.051538,0.666155,-0.507304,0.695088,1.374029,0.096443,-0.073810
3,-0.100354,-0.196462,0.234995,0.381653,-1.669789,-0.064474,-0.453599,-0.008013,-0.583915,-0.347269,...,0.676468,0.003009,-0.709290,0.613343,-0.100473,0.578137,0.556590,-1.099979,-0.006314,-1.389844
4,1.167685,1.636609,1.395270,0.978650,-0.122622,0.780046,0.984550,0.421005,1.467246,2.011248,...,0.973355,1.395133,-0.308862,0.166082,-0.143099,1.507798,0.843175,1.204872,1.041953,-0.209867
5,-1.126979,-0.438830,0.179063,-0.158883,1.491291,-1.028443,-0.911609,-1.613785,-0.085055,-0.776853,...,-0.409763,-0.056387,0.250051,0.999753,0.943649,0.474284,0.042748,-0.311495,0.704315,-0.873573


In [31]:
final_df = pd.concat([track_ids, firsts_zscore, categoricals, lasts_zscore], axis=1)
final_df.head(6)

,track_id,duration_zscore,fade_in_zscore,tempo_zscore,energy_zscore,danceability_zscore,loudness_zscore,mode,0,2,...,timbre_mode3_zscore,timbre_mode4_zscore,timbre_mode5_zscore,timbre_mode6_zscore,timbre_mode7_zscore,timbre_mode8_zscore,timbre_mode9_zscore,timbre_mode10_zscore,timbre_mode11_zscore,timbre_mode12_zscore
0,TRBGYHC12903D0626A,1.932092,-0.442154,1.619931,0,0,0.328287,1,1,0,...,1.189369,0.902684,0.114042,-0.282640,0.677379,-0.392908,-0.017701,-0.004100,0.248583,1.196521
1,TRBHZRN128F92EFF91,-0.348405,-0.347275,0.957805,0,0,-0.145635,0,0,0,...,-1.039499,-1.720896,-1.271733,0.555001,-2.043612,-1.660007,-2.119900,-1.163327,-2.084980,1.350574
2,TRBFXLH12903CBAFAD,-0.934328,-0.449523,-0.732683,0,0,-0.545705,0,0,0,...,-1.389931,-0.523542,1.925792,-2.051538,0.666155,-0.507304,0.695088,1.374029,0.096443,-0.073810
3,TRAZFZS128F4272A31,-0.270476,2.232878,-0.969715,0,0,-1.607392,0,0,0,...,0.676468,0.003009,-0.709290,0.613343,-0.100473,0.578137,0.556590,-1.099979,-0.006314,-1.389844
4,TRAGYJM128EF3466D2,-0.940352,-0.529664,0.109211,0,0,1.709166,1,0,1,...,0.973355,1.395133,-0.308862,0.166082,-0.143099,1.507798,0.843175,1.204872,1.041953,-0.209867
5,TRAFJKG128F9332800,0.561469,-0.464262,-0.984549,0,0,0.261279,1,0,0,...,-0.409763,-0.056387,0.250051,0.999753,0.943649,0.474284,0.042748,-0.311495,0.704315,-0.873573
